In [1]:
import pandas as pd
import time
import threading

### Simple Example With Two Functions Started in Parallel

In [2]:
def func_a():
    print("func_a started")
    time.sleep(3)
    print("func_a finished")

def func_b():
    print("func_b started")
    time.sleep(3)
    print("func_b finished")
    
threading.Thread(target=func_a).start()
threading.Thread(target=func_b).start()

func_a started
func_b started


In [3]:
time.sleep(5)

func_a finished
func_b finished


### Drone Emulation

In [4]:
df = pd.read_csv('../../data/posture/posture_training_data.csv')

interval = 0.130
drone_last_action = time.time()
drone_status = 'grounded'
movements = {0: 'takeoff',
             1: 'move_forward',
             2: 'flip',
             3: 'rotate_cw',
             4: 'rotate_ccw',
             5: 'land',
             999: 'not detected'}

In [5]:
df.sample(3)

,leftShoulder_x,leftShoulder_y,rightShoulder_x,rightShoulder_y,leftWrist_x,leftWrist_y,rightWrist_x,rightWrist_y,leftHip_x,leftHip_y,rightHip_x,rightHip_y,label
899,273.643012,174.031938,223.671055,171.372575,355.393601,159.416145,144.867507,167.850433,267.015255,255.516243,235.386515,252.927435,2
68,369.905967,161.509110,325.248454,154.171085,383.333624,242.164282,243.211328,143.941467,365.549369,237.659162,332.829002,239.078635,4
224,337.397562,138.261967,300.869204,141.371509,337.336890,65.369459,296.043133,64.239784,334.968510,230.549861,306.783627,231.192493,1


In [6]:
def steer_drone(movement):
    global drone_last_action
    global drone_status
    if (time.time() - drone_last_action) > 1.5:
        drone_last_action = time.time()
        if ((movement == 0) & (drone_status == 'grounded')):
            threading.Thread(target=drone_takeoff).start()
        if (drone_status != 'grounded'):
            if movement == 1:
                threading.Thread(target=drone_move_forward).start()
            if movement == 2:
                threading.Thread(target=drone_flip).start()
            if movement == 3:
                threading.Thread(target=drone_rotate_cw).start()
            if movement == 4:
                threading.Thread(target=drone_rotate_ccw).start()
            if movement == 5:
                threading.Thread(target=drone_land).start()

In [7]:
def predict_movement(pose):

    movement = 999

    leftArm_x = pose['leftWrist_x'] - pose['leftShoulder_x']
    rightArm_x = pose['rightShoulder_x'] - pose['rightWrist_x']
    leftArm_y = pose['leftShoulder_y'] - pose['leftWrist_y']
    rightArm_y = pose['rightShoulder_y'] - pose['rightWrist_y']

    if ((leftArm_y > 100) & (rightArm_y > 100) & (abs(leftArm_x) < 30) & (abs(rightArm_x) < 30)):
        movement = 0 # takeoff
    
    if ((abs(leftArm_y) < 30) & (abs(rightArm_y) < 30) & (leftArm_x > 60) & (rightArm_x > 60)):
        movement = 1 # move_forward

    if ((abs(leftArm_x) < 30) & (abs(rightArm_x) < 30) & (abs(leftArm_y) < 30) & (abs(rightArm_y) < 30)):
        movement = 2 # flip

    if ((leftArm_y < -100) & (abs(rightArm_y) < 30) & (abs(leftArm_x) < 30) & (rightArm_x > 60)):
        movement = 3 # rotate_cw

    if ((abs(leftArm_y) < 30) & (rightArm_y < -100) & (leftArm_x > 60) & (abs(rightArm_x) < 30)):
        movement = 4 # rotate_ccw

    if ((leftArm_y < -100) & (rightArm_y < -100) & (abs(leftArm_x) < 30) & (abs(rightArm_x) < 30)):
        movement = 5 # land

    return movement

In [8]:
def drone_takeoff():
    global drone_status
    drone_status = 'flying'
    print('drone.takeoff()')
    print("drone_status = 'flying'")
    print('time.sleep(5)')
    time.sleep(5)

In [9]:
def drone_move_forward():
    print('drone.move_forward(2)')
    time.sleep(1)

In [10]:
def drone_flip():
    print("drone.flip('r')")
    time.sleep(1)

In [11]:
def drone_rotate_cw():
    print('drone.rotate_cw(45)')
    time.sleep(1)

In [12]:
def drone_rotate_ccw():
    print('drone.rotate_ccw(45)')
    time.sleep(1)

In [13]:
def drone_land():
    global drone_status
    drone_status = 'grounded'
    print('drone.land()')
    print("drone_status = 'grounded'")

In [14]:
steer_drone(0)

drone.takeoff()
drone_status = 'flying'
time.sleep(5)


In [15]:
time.sleep(5)

In [16]:
for index, pose in df.iterrows():
    steer_drone(predict_movement(pose))
    time.sleep(interval)
    if (index > 100):
        break

drone.flip('r')
drone.move_forward(2)
drone.flip('r')
drone.flip('r')
